In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns
from statannotations.Annotator import Annotator

# enables concurrent editing of base.py
from importlib import reload
reload(base)

In [ ]:
# Load parameter sweeps from ODE model
simulation_path = rd.rootdir/'data'/'slope_sweep_df.gzip'
data = pd.DataFrame()
if simulation_path.is_file(): 
    data = pd.read_parquet(simulation_path)
display(data)

In [ ]:
# Normalize parameter values such that original (middle) value = 1
data = data.groupby('param')[data.columns].apply(base.normalize_param_val).reset_index(drop=True)

# Compute value of unregulated gene
alpha_rna = 4.67e-2     # params from `miR_iFFL.jl`
delta_mrna = 2.88e-4
alpha_p = 3.33e-4
delta_p = 9.67e-5
data['unreg'] = data['copy_num'] * (alpha_rna * alpha_p) / (delta_mrna * delta_p)

# Compute instantaneous slope for each param & param_val 
col_list = ['copy_num','protein']
slopes = data.groupby(['param','param_val','param_val_norm'])[data.columns].apply(lambda x: base.get_slope_instant(x, *col_list)).rename('slope').reset_index()
slopes['base_norm_factor'] = (delta_mrna * delta_p) / (alpha_rna * alpha_p)

result = slopes.groupby(['param','param_val_norm'])[slopes.columns].apply(base.modify_norm_factor).rename('norm_factor').reset_index()
slopes['norm_factor'] = result['norm_factor']
slopes['slope_norm'] = slopes['slope'] * slopes['norm_factor']
slopes['gene'] = 'output'

# Compute slope for unregulated gene -> delta slope
by = ['param','param_val']
slopes_unreg = data.groupby(by)[data.columns].apply(lambda x: base.get_slope_instant(x, *['copy_num','unreg'])).rename('slope').reset_index()
slopes_unreg['norm_factor'] = result['norm_factor']
slopes_unreg['slope_norm'] = slopes_unreg['slope'] * slopes_unreg['norm_factor']
slopes_unreg['gene'] = 'marker'

delta_slopes = slopes.merge(slopes_unreg, on=by, suffixes=['_output','_marker'])
delta_slopes['delta_slope'] = delta_slopes['slope_norm_marker'] - delta_slopes['slope_norm_output']
delta_slopes.drop(columns=['base_norm_factor','norm_factor_output','gene_output','norm_factor_marker','gene_marker'], inplace=True)
slopes = pd.concat([slopes, slopes_unreg])

display(slopes)
display(delta_slopes)

In [ ]:
delta_slopes['slope_ratio'] = delta_slopes['slope_norm_output'] / delta_slopes['slope_norm_marker']

In [ ]:
f = sns.relplot(data=slopes, x='param_val_norm', y='slope_norm', col='param',
                kind='line', col_wrap=3, hue='gene')
f.set(xscale='log')

In [ ]:
display(slopes.param.unique())

In [ ]:
params = ['α_im','α_p','δ_m','δ_p']
plot_df = slopes[slopes.param.isin(params)]
f = sns.relplot(data=plot_df, x='param_val_norm', y='slope_norm', col='param',
                kind='line', col_wrap=2, hue='gene')
f.set(xscale='log')

In [ ]:
params = ['k_deg','k_mRNA_bind','k_mRNA_unbind','Rtot']
plot_df = slopes[slopes.param.isin(params)]
f = sns.relplot(data=plot_df, x='param_val_norm', y='slope_norm', col='param',
                kind='line', col_wrap=3, hue='gene')
f.set(xscale='log')

In [ ]:
params = ['α_im','α_p','δ_m','δ_p']
plot_df = delta_slopes[delta_slopes.param.isin(params)]
f = sns.lineplot(data=plot_df, x='param_val_norm', y='delta_slope', hue='param',)
f.set(xscale='log')
f.axhline(1, color='black', ls=':', zorder=0)

In [ ]:
params = ['k_deg','k_mRNA_bind','k_mRNA_unbind','Rtot']
plot_df = delta_slopes[delta_slopes.param.isin(params)]
f = sns.lineplot(data=plot_df, x='param_val_norm', y='delta_slope', hue='param',)
f.set(xscale='log')
f.axhline(1, color='black', ls=':', zorder=0)

In [ ]:
params = ['k_deg','k_mRNA_bind','Rtot','α_im','α_p']
plot_df = delta_slopes[delta_slopes.param.isin(params)]
f = sns.lineplot(data=plot_df, x='param_val_norm', y='delta_slope', hue='param',)
f.set(xscale='log')
f.axhline(1, color='black', ls=':', zorder=0)

### Compare to data

Load lenti data for all cell types (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'lenti')

In [ ]:
# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
subset = stats[((stats['cell']=='neuron') & (stats['moi']==7) & ~(stats['name'].str.contains('FXN')) & ~(stats['name'].str.contains('FMRP'))) |
                 ((stats['cell'].isin(['MEF','tcell','iPS11'])) & (stats['dox']==1000) & (stats['moi']==1) & (stats['group'].isin(['base','controller'])) & (stats['design']<2))]
subset = subset[['cell','biorep','ts_kind','slope']]

subset_T = subset[subset.ts_kind=='T']
subset_NT = subset[subset.ts_kind=='NT']
data_slopes = subset_NT.merge(subset_T, on=['cell','biorep'], suffixes=['_NT','_T'])
data_slopes['delta_slope'] = data_slopes['slope_NT'] - data_slopes['slope_T']
display(data_slopes)


In [ ]:
f = sns.pointplot(data=data_slopes, x='cell', y='delta_slope',
                  **base.scatter_kwargs, order=['neuron','MEF','tcell','iPS11'])
f.set(ylim=(0,1))

In [ ]:
fig, axes = plt.subplots(1,5, figsize=(15,3), gridspec_kw=dict(wspace=0.4))
ax = axes[0]
params = ['k_deg','k_mRNA_bind','Rtot']
for param in params:
    plot_df = slopes[slopes.param==param]
    sns.lineplot(data=plot_df, x='param_val_norm', y='slope_norm', hue='gene', ax=ax,
                 legend=False, palette={'output': base.colors['teal'], 'marker': base.colors['black']})
ax.set(xscale='log', xlabel='Norm. parameter', ylabel='slope')

ax = axes[1]
plot_df = delta_slopes[delta_slopes.param.isin(params)]
sns.lineplot(data=plot_df, x='param_val_norm', y='slope_ratio', hue='param', ax=ax,)
ax.set(xscale='log', xlabel='Norm. parameter', ylabel='slope ratio')

ax = axes[2]
params = ['α_im','α_p']
for param in params:
    plot_df = slopes[slopes.param==param]
    sns.lineplot(data=plot_df, x='param_val_norm', y='slope_norm', hue='gene', ax=ax,
                 legend=False, palette={'output': base.colors['teal'], 'marker': base.colors['black']})
ax.set(xscale='log', xlabel='Norm. parameter', ylabel='slope')

ax = axes[3]
plot_df = delta_slopes[delta_slopes.param.isin(params)]
sns.lineplot(data=plot_df, x='param_val_norm', y='slope_ratio', hue='param', ax=ax)
ax.set(xscale='log', xlabel='Norm. parameter', ylabel='slope ratio')

ax = axes[4]
sns.pointplot(data=data_slopes, x='cell', y='delta_slope', ax=ax,
              **base.scatter_kwargs, order=['neuron','MEF','tcell','iPS11'])
ax.set(ylim=(0,1), ylabel='$\Delta$ slope')

for ax in axes: sns.despine(ax=ax)

fig.savefig(rd.outfile(rd.rootdir/'output'/'more-modeling'/'summary.png'))

In [ ]:
data_slopes['slope_ratio'] = data_slopes['slope_T'] / data_slopes['slope_NT']

In [ ]:
g = sns.pointplot(data=data_slopes, x='cell', y='slope_ratio',
              **base.scatter_kwargs, order=['neuron','MEF','tcell','iPS11'])
g.set(ylim=(0,1), ylabel='slope ratio')